<a href="https://colab.research.google.com/github/BharatsDruv/hackindore2.0/blob/master/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [0]:
! git clone https://github.com/BharatsDruv/hackindore2.0.git

Cloning into 'hackindore2.0'...
remote: Enumerating objects: 2534, done.
remote: Counting objects: 100% (2534/2534), done.
remote: Compressing objects: 100% (2533/2533), done.
remote: Total 2534 (delta 0), reused 2534 (delta 0), pack-reused 0
Receiving objects: 100% (2534/2534), 40.43 MiB | 9.64 MiB/s, done.


In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(6,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [0]:
model=Model(inputs=base_model.input,outputs=preds)

In [0]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('hackindore2.0/dataset-resized', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 2527 images belonging to 6 classes.


In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

Epoch 1/5
78/78 [==============================] - 27s 350ms/step - loss: 0.8667 - acc: 0.7022
Epoch 2/5
78/78 [==============================] - 17s 215ms/step - loss: 0.4450 - acc: 0.8645
Epoch 3/5
78/78 [==============================] - 17s 214ms/step - loss: 0.3639 - acc: 0.8794
Epoch 4/5
78/78 [==============================] - 17s 213ms/step - loss: 0.2635 - acc: 0.9163
Epoch 5/5
78/78 [==============================] - 17s 213ms/step - loss: 0.2985 - acc: 0.9130


In [0]:
import pickle

In [0]:
model.save("model.h5")